# Ring network with MPI using Arbor's python interface

This example shows the usage of mpi in arbor's python backend by using helpers provided by our Python wrapper in a ring network. 
    
To do so, we first have to load our arbor python module 'pyarb' and submodules thereof:

In [1]:
%matplotlib notebook
import sys
import numpy as np
import math
import matplotlib.pyplot as plot

import pyarb as arb
from pyarb import connection
from pyarb import cell_member as cmem

## Create a recipe for ring network

Now, a recipe, that describes the cells and network of a model, can be defined in python by implementing the recipe interface: 

    1) constructor setting the number of cells
    2) function returning the total number of cells in the model
    3) function returning a cell description (mechanism) of soma as well as segment, synapse and stimulus locations 
    4) function returning the number of targets (here: 1 connection to next cell) 
    5) function returning the number of sources (here: 1 connenction from previous cell)
    6) function returning the type of cell with global id gid 
    7) function creating the ring network (incoming connections to each cell)

In [2]:
class ring_recipe(arb.recipe):
    
    def __init__(self, n=4):
        # The base C++ class constructor must be called first, to ensure that
        # all memory in the C++ class is initialized correctly.
        arb.recipe.__init__(self)
        self.ncells = n
    
    # The num_cells method that returns the total number of cells in the model
    # must be implemented.
    def num_cells(self):
        return self.ncells
    
    # The cell_description method returns a cell
    def cell_description(self, gid):
        
        cell = arb.make_soma_cell()
        loc = arb.segment_location(0, 0.5)
        cell.add_synapse(loc)
        cell.add_detector(loc, 20)
        
        if gid==0:
            cell.add_stimulus(loc, 0, 20, 0.01)
        return cell
    
    def num_targets(self, gid):
        return 1
    
    def num_sources(self, gid):
        return 1
    
    # The kind method returns the type of cell with gid.
    # Note: this must agree with the type returned by cell_description.
    def kind(self, gid):
        return arb.cell_kind.cable1d
    
    # Make a ring network
    def connections_on(self, gid):
        src = self.num_cells()-1 if gid==0 else gid-1
        return [connection(cmem(src,0), cmem(gid,0), 0.1, 10)]

## Create parallel execution context

First, initialize MPI with Arbor's mpi helpers:

In [3]:
arb.mpi_init();

Then, get MPI communicator:

In [4]:
comm = arb.mpi_comm()

print(comm)

<mpi_comm: MPI_COMM_WORLD>


Then, get the resources and create the parallel execution context

In [5]:
resources = arb.proc_allocation()

context = arb.context(resources, comm)

print(context, '\n')

<context: threads 24, gpu yes, distributed MPI ranks 1> 



Here, we have 24 threads, a P100 GPU and one local rank.

### Set up measurements for runtime and consumption
In order to measure the runtime, the memory consumption on CPU and GPU, as well as the energy consumption, we can utilize arbor's meter manager and start the measurement: 

In [6]:
# OPTIONAL            # ================================ TASK 4 ================================ #
# OPTIONAL            # 4a) Initiate meter manager and start the measurement
meters = #TODO#
#TODO#

SyntaxError: invalid syntax (<ipython-input-6-a7246efe2bfb>, line 3)

### Initiate ring network

Next, we create the example ring network with 100 cells and set a checkpoint to measure its creation:

In [7]:
n_cells = 100
recipe = ring_recipe(n_cells)

In [8]:
# OPTIONAL            # 4b) i) Set checkpoint 'recipe create'
#TODO#

### Partition the simulation over the distributed system

To part the work into segments, we decompose the recipe (and measure again):

In [9]:
decomp = arb.partition_load_balance(recipe, context)

In [10]:
# OPTIONAL            # 4b) ii) Set checkpoint 'load balance'
#TODO#

## Build and run the simulation 

Next, we build the simulation, initialize a spike recorder (and measure the simulation initialization):

In [11]:
sim = arb.simulation(recipe, decomp, context)

In [12]:
# OPTIONAL            # 4b) iii) Set checkpoint 'simulation init'
#TODO#

# OPTIONAL            # 4c) Build the spike recorder
#TODO

### Run simulation 

Finally, we can run the simulation for 2000 ms and with time step size 0.025 ms (and measure the simulation run):

In [13]:
tSim = 2000
dt = 0.025
sim.run(tSim, dt)

In [14]:
# OPTIONAL            # 4b) iv) Set checkpoint 'simulation run'
#TODO#

## Analyze results of meter measurement and spike recordings

After the simulation run is completed, we print the information collected with arbor's meter manager.

In [15]:
# OPTIONAL            # 4d) Make and print meter report
#TODO#

With the spike recorder we can now print the occuring spike times at correcponding cells:

In [16]:
print('SPIKES:')

# OPTIONAL            # 4e) Get the recorder`s spikes
spikes = #TODO#

# print at most 10 spikes
n_spikes_out = min(len(spikes), 10)
for i in range(n_spikes_out):
    spike = spikes[i]
    print('  cell %2d at %8.3f ms'%(spike.source.gid, spike.time))



if n_spikes_out<len(spikes):
    print('  ...')
    spike = spikes[-1]
    print('  cell %2d at %8.3f ms'%(spike.source.gid, spike.time))

SyntaxError: invalid syntax (<ipython-input-16-6b34b62b1624>, line 4)

# Visualization of spiking activity

Use a raster plot to visualize spiking activity.

In [ ]:
n_spikes = len(spikes)
tVec = np.arange(0,tSim,dt)
SpikeMat_rows = n_cells # number of cells
SpikeMat_cols = math.floor(tSim/dt)
SpikeMat = np.zeros((SpikeMat_rows, SpikeMat_cols))

# save spike trains in matrix: 
# (if spike in cell n at time step k, then SpikeMat[n,k]=1, else 0)
for i in range(n_spikes):
    spike = spikes[i]
    tCur = math.floor(spike.time/dt)
    SpikeMat[spike.source.gid][tCur] = 1

for i in range(SpikeMat_rows):
    for j in range(SpikeMat_cols):
        if(SpikeMat[i,j] == 1):
            x1 = [i,i+0.5]
            x2 = [j,j]
            plot.plot(x2,x1,color = 'black')

plot.title('Spike raster plot')
plot.xlabel('Spike time (ms)')
tick = range(0,SpikeMat_cols+10000,10000)
label = range(0,tSim+250,250)
plot.xticks(tick, label)
plot.ylabel('Neuron (gid)')
plot.show()

## Finalize MPI with Arbor's mpi helper

In [17]:
arb.mpi_finalize();